In [46]:
import sys, os
print(sys.executable)
print(os.getcwd())

C:\Users\chenr\miniforge3\envs\astropy\python.exe
C:\Users\chenr\OneDrive\Desktop\phys-134-proj\lco_data-20251202-36


In [47]:
rootfolder=os.getcwd().replace('\\','/')
print(rootfolder)

C:/Users/chenr/OneDrive/Desktop/phys-134-proj/lco_data-20251202-36


In [48]:
os.chdir(f"{rootfolder}/lco_data-20251202-36")
os.getcwd()

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'C:/Users/chenr/OneDrive/Desktop/phys-134-proj/lco_data-20251202-36/lco_data-20251202-36'

In [49]:
import pickle 
with open('region_data.pkl', 'rb') as f:
    region_data = pickle.load(f)

In [50]:
file_list=os.listdir(".")

# filter to only .fits files
file_list=[f for f in file_list if f.endswith('.fits') and f.startswith('2025')]
print(file_list)

['2025-11-25-h02-m00-s14-ms936-tfn.fits', '2025-11-25-h05-m00-s13-ms688-lsc.fits', '2025-11-25-h05-m36-s42-ms454-lsc.fits', '2025-11-25-h06-m06-s14-ms909-lsc.fits', '2025-11-25-h07-m45-s12-ms840-elp.fits', '2025-11-25-h08-m30-s09-ms718-elp.fits', '2025-11-25-h09-m06-s12-ms841-elp.fits', '2025-11-25-h10-m06-s14-ms367-elp.fits', '2025-11-25-h11-m06-s18-ms072-elp.fits', '2025-11-25-h12-m47-s39-ms526-ogg.fits', '2025-11-25-h13-m06-s13-ms287-ogg.fits', '2025-11-25-h14-m06-s15-ms196-ogg.fits', '2025-11-25-h23-m02-s40-ms929-tfn.fits', '2025-11-25-h23-m35-s17-ms210-cpt.fits', '2025-11-26-h00-m55-s17-ms907-tfn.fits', '2025-11-26-h01-m55-s17-ms940-tfn.fits', '2025-11-26-h02-m10-s07-ms425-tfn.fits', '2025-11-26-h03-m06-s19-ms581-tfn.fits', '2025-11-26-h04-m06-s16-ms133-tfn.fits', '2025-11-26-h05-m35-s14-ms393-lsc.fits', '2025-11-26-h06-m06-s06-ms084-lsc.fits', '2025-11-26-h07-m59-s44-ms591-elp.fits', '2025-11-26-h08-m06-s06-ms512-elp.fits', '2025-11-26-h11-m06-s14-ms445-ogg.fits', '2025-11-26-h12

In [51]:
# region data:
#  filename (no extensions, strip[0] '.')
#     x: []
#     y: []
# where x[i], y[i] are the pixel coordinates of reference star i in that file

# For each observation file:
#     Load entire image array
#     For each reference star in region_data: (from pickle): 
#      Extract small cutout around star (+- 20 pixels square from center)
#      Use function to fit 2D Gaussian on that cutout to get centroid and sigma -> store in array 
#           whose index matches reference star index in region_data
#           and whose keys are: 'x_cen', 'y_cen', 'sigma_xy'     
#     Designate apertures and annuli based on x_cen, y_cen, where app_rad=7*sigma_xy, ann_rad_in=9*sigma_xy, ann_rad_out=11*sigma_xy

#     Calculate total flux subtracted by background, using 3-sigma medium clip on annulus pixels to get background level
#     Store flux in array whose index matches reference star index in region_data
#
#     Generate an tile plot for this observation file, showing each reference star with aperture and annulus overlaid
# At end, save all flux arrays to file using pickle



In [52]:
center= 20

In [53]:
from scipy.optimize import curve_fit
def gaussian_2d(xy, amp, xo, yo, sigma, offset):
    x, y = xy
    g = offset + amp * np.exp(-(((x - xo) ** 2 + (y - yo) ** 2) / (2 * sigma ** 2)))
    return g.ravel()

In [65]:
from astropy.io import fits
import numpy as np

# cutouts: dict(key: filename, value: list of cutouts for that file)
cutouts_obs = {}
init_guesses_obs = {}

for idx, filename in enumerate(file_list):
    print(f"Processing file {idx+1} of {len(file_list)}: {filename}")
    hdul = fits.open(filename)
    sci_data = hdul['SCI'].data
    hdul.close() #we can load the uncertainty later for error analysis
    
    x_centers = region_data[filename]['x']
    y_centers = region_data[filename]['y']
    
    x_starts = [round(xc) - center for xc in x_centers]
    y_starts = [round(yc) - center for yc in y_centers]
    
    x_ends = [round(xc) + center+1 for xc in x_centers]
    y_ends = [round(yc) + center+1 for yc in y_centers]
    
    cutouts = [sci_data[ys:ye, xs:xe] for xs, xe, ys, ye in zip(x_starts, x_ends, y_starts, y_ends)]
    init_guesses = [(cutout.max() - cutout.min(), center, center, 4, cutout.min()) for cutout in cutouts]
    # reference star is too close to edge on file 7! error here
    
    cutouts_obs[filename] = cutouts
    init_guesses_obs[filename] = init_guesses

Processing file 1 of 36: 2025-11-25-h02-m00-s14-ms936-tfn.fits
Processing file 2 of 36: 2025-11-25-h05-m00-s13-ms688-lsc.fits
Processing file 3 of 36: 2025-11-25-h05-m36-s42-ms454-lsc.fits
Processing file 4 of 36: 2025-11-25-h06-m06-s14-ms909-lsc.fits
Processing file 5 of 36: 2025-11-25-h07-m45-s12-ms840-elp.fits
Processing file 6 of 36: 2025-11-25-h08-m30-s09-ms718-elp.fits
Processing file 7 of 36: 2025-11-25-h09-m06-s12-ms841-elp.fits
Processing file 8 of 36: 2025-11-25-h10-m06-s14-ms367-elp.fits
Processing file 9 of 36: 2025-11-25-h11-m06-s18-ms072-elp.fits
Processing file 10 of 36: 2025-11-25-h12-m47-s39-ms526-ogg.fits
Processing file 11 of 36: 2025-11-25-h13-m06-s13-ms287-ogg.fits
Processing file 12 of 36: 2025-11-25-h14-m06-s15-ms196-ogg.fits
Processing file 13 of 36: 2025-11-25-h23-m02-s40-ms929-tfn.fits
Processing file 14 of 36: 2025-11-25-h23-m35-s17-ms210-cpt.fits
Processing file 15 of 36: 2025-11-26-h00-m55-s17-ms907-tfn.fits
Processing file 16 of 36: 2025-11-26-h01-m55-s17-

In [66]:
cutoutex = cutouts_obs[file_list[0]][0]
cutoutex_shape = cutoutex.shape
x = np.linspace(0, cutoutex_shape[1] - 1, cutoutex_shape[1])
y = np.linspace(0, cutoutex_shape[0] - 1, cutoutex_shape[0])
x, y = np.meshgrid(x, y)

In [67]:
# Image processing function (debugging)
def imageprocessing(arr, min_clip=-100, max_clip=8000, a=1000, contrast=3.86, bias=0.442):
    clipped = np.clip(arr, min_clip, max_clip)
    norm = (clipped - min_clip) / (max_clip - min_clip)
    log = np.log(1 + a*norm) / np.log(a)
    final_data = (log- 0.5) * contrast + bias
    final_data = np.clip(final_data, 0, 1)
    return final_data

In [72]:
from photutils.aperture import CircularAnnulus, CircularAperture, ApertureStats
import matplotlib.pyplot as plt
from astropy.stats import SigmaClip

reference_star_fluxes_reduced = {}

for idx, filename in enumerate(file_list):
    print(f"Fitting Gaussians for file {idx+1} of {len(file_list)}: {filename}")
    cutouts = cutouts_obs[filename]
    init_guesses = init_guesses_obs[filename]
    
    gaussian_fits = []
    for idx_cutout, cutout in enumerate(cutouts):
        try:
            popt, pcov = curve_fit(gaussian_2d, (x, y), cutout.ravel(), p0=init_guesses[idx_cutout])
        except RuntimeError as e:
            print(f"RuntimeError for cutout {idx_cutout} in file {idx}: {filename}: {e}")
            cutouts.pop(idx_cutout)
            init_guesses.pop(idx_cutout)
            continue
        
        if popt[3] <= 0:
            print(f"Warning: Removed non-physical sigma value {popt[3]} for cutout {idx_cutout} in file {idx}: {filename}")
        
            # there was probably an error on the centroid estimation 
            # remove this index from cutouts and init_guesses for this file
            cutouts.pop(idx_cutout)
            init_guesses.pop(idx_cutout)
            continue
            
        gaussian_fits.append(popt)
            #visualize
            
            # import matplotlib.pyplot as plt
            
            # plt.imshow(imageprocessing(cutout), cmap='gray', origin='lower', interpolation='nearest')
            # #overlay centroid on cutout
            # xo_fit, yo_fit = popt[1], popt[2]
            # plt.scatter(xo_fit, yo_fit, color='red', marker='x', s=100, label='Fitted Centroid')
            # plt.title(f'Cutout {idx_cutout} with Fitted Centroid')
            # plt.legend()
            # plt.show()
        
        #pcov contains covariance matrix for errors, can use later for error analysis
        
    positions = [(popt[1], popt[2]) for popt in gaussian_fits]
    apertures = [CircularAperture(pos, r=7*popt[3]) for pos, popt in zip(positions, gaussian_fits)]
    annulus_apertures = [CircularAnnulus(pos, r_in=9*popt[3], r_out=11*popt[3]) for pos, popt in zip(positions, gaussian_fits)]
    

    sigclip = SigmaClip(sigma=3.0, maxiters=10)
    
    aper_stats_list = [ApertureStats(cutout, aperture=ap, sigma_clip=None) 
                       for cutout, ap in zip(cutouts, apertures)]
    annulus_stats_list = [ApertureStats(cutout, aperture=ann_ap, sigma_clip=sigclip) 
                          for cutout, ann_ap in zip(cutouts, annulus_apertures)]
    
    total_anulus_list = [annulus_stats.median * aper_stats.sum_aper_area.value for annulus_stats, aper_stats in zip(annulus_stats_list, aper_stats_list)]
    
    apersum_subanulus_list  = [aper_stats.sum - total_anulus 
                               for aper_stats, total_anulus in zip(aper_stats_list, total_anulus_list)]
    reference_star_fluxes_reduced[filename] = apersum_subanulus_list

Fitting Gaussians for file 1 of 36: 2025-11-25-h02-m00-s14-ms936-tfn.fits
Fitting Gaussians for file 2 of 36: 2025-11-25-h05-m00-s13-ms688-lsc.fits
Fitting Gaussians for file 3 of 36: 2025-11-25-h05-m36-s42-ms454-lsc.fits
Fitting Gaussians for file 4 of 36: 2025-11-25-h06-m06-s14-ms909-lsc.fits
Fitting Gaussians for file 5 of 36: 2025-11-25-h07-m45-s12-ms840-elp.fits
Fitting Gaussians for file 6 of 36: 2025-11-25-h08-m30-s09-ms718-elp.fits
Fitting Gaussians for file 7 of 36: 2025-11-25-h09-m06-s12-ms841-elp.fits
Fitting Gaussians for file 8 of 36: 2025-11-25-h10-m06-s14-ms367-elp.fits
Fitting Gaussians for file 9 of 36: 2025-11-25-h11-m06-s18-ms072-elp.fits
Fitting Gaussians for file 10 of 36: 2025-11-25-h12-m47-s39-ms526-ogg.fits
Fitting Gaussians for file 11 of 36: 2025-11-25-h13-m06-s13-ms287-ogg.fits
Fitting Gaussians for file 12 of 36: 2025-11-25-h14-m06-s15-ms196-ogg.fits
Fitting Gaussians for file 13 of 36: 2025-11-25-h23-m02-s40-ms929-tfn.fits
Fitting Gaussians for file 14 of 3

C:\Users\chenr\AppData\Local\Temp\ipykernel_64228\168779292.py:15: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, pcov = curve_fit(gaussian_2d, (x, y), cutout.ravel(), p0=init_guesses[idx_cutout])


Fitting Gaussians for file 21 of 36: 2025-11-26-h06-m06-s06-ms084-lsc.fits
Fitting Gaussians for file 22 of 36: 2025-11-26-h07-m59-s44-ms591-elp.fits
Fitting Gaussians for file 23 of 36: 2025-11-26-h08-m06-s06-ms512-elp.fits
Fitting Gaussians for file 24 of 36: 2025-11-26-h11-m06-s14-ms445-ogg.fits
Fitting Gaussians for file 25 of 36: 2025-11-26-h12-m06-s12-ms274-ogg.fits
Fitting Gaussians for file 26 of 36: 2025-11-26-h13-m06-s14-ms392-ogg.fits
Fitting Gaussians for file 27 of 36: 2025-11-26-h14-m06-s13-ms681-ogg.fits
Fitting Gaussians for file 28 of 36: 2025-11-26-h15-m14-s48-ms233-coj.fits
Fitting Gaussians for file 29 of 36: 2025-11-26-h23-m00-s14-ms461-cpt.fits
Fitting Gaussians for file 30 of 36: 2025-11-26-h23-m35-s15-ms525-cpt.fits
Fitting Gaussians for file 31 of 36: 2025-11-27-h04-m59-s38-ms384-tfn.fits
Fitting Gaussians for file 32 of 36: 2025-11-27-h05-m06-s21-ms675-tfn.fits
Fitting Gaussians for file 33 of 36: 2025-11-27-h07-m44-s42-ms254-elp.fits
Fitting Gaussians for fil

In [77]:
reference_star_fluxes_reduced[list(reference_star_fluxes_reduced.keys())[0]]
#visualize in 2d arra
import pandas as pd
reference_star_fluxes_df = pd.DataFrame.from_dict(reference_star_fluxes_reduced, orient='index')
reference_star_fluxes_df.head()

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,41,42,43
2025-11-25-h02-m00-s14-ms936-tfn.fits,4753.716110,9066.523540,4092.431992,3126.404280,4893.403818,5220.404590,5969.595399,10193.417278,5310.325080,5702.784839,...,3816.864742,7875.412813,15262.066397,5307.408264,18150.725424,7048.983816,20268.961141,10534.690782,2840.039459,4232.726976
2025-11-25-h05-m00-s13-ms688-lsc.fits,4820.177015,9714.751992,4307.416315,3377.810027,5046.425381,5519.132304,6066.418422,10504.053591,5492.196347,6068.372608,...,4018.255968,8541.657043,15585.575318,5465.270323,18553.502916,7625.772556,21462.247444,10548.614510,2828.836923,4292.963258
2025-11-25-h05-m36-s42-ms454-lsc.fits,5064.517392,9568.482848,4543.174628,3031.169916,5129.422111,5201.729006,6336.517693,10754.529680,5603.326682,6193.333913,...,3974.589887,8546.748137,16641.340812,5634.537765,18967.690049,7510.760594,22042.632047,10851.367556,2796.326917,NaN
2025-11-25-h06-m06-s14-ms909-lsc.fits,5078.455791,9672.271613,4478.482298,3320.739723,4920.541988,5358.440967,6043.621453,10550.034800,5516.790204,6191.375479,...,4036.146346,8548.993066,15963.384457,5448.630457,19119.415244,7499.300281,21583.191185,10808.248459,2837.898192,4237.174094
2025-11-25-h07-m45-s12-ms840-elp.fits,4260.196058,8442.304461,3794.923821,2829.785415,4524.402060,4963.327059,5317.218639,9559.167553,4742.408770,5422.043372,...,7498.152916,13888.321649,4822.047688,16258.890840,6533.184494,19020.230571,2462.077573,NaN,NaN,NaN


In [88]:
reference_star_fluxes_reduced['2025-11-26-h04-m06-s16-ms133-tfn.fits']

[np.float64(517.9552321122941),
 np.float64(948.2806012084059),
 np.float64(530.0071761735474),
 np.float64(134.8149211147586),
 np.float64(480.3805890216473),
 np.float64(518.1685091326069),
 np.float64(674.4929260471506),
 np.float64(1081.76121278342),
 np.float64(402.87780988261994),
 np.float64(800.7342232457179),
 np.float64(681.2827564748709),
 np.float64(641.510194451901),
 np.float64(589.5182588574883),
 np.float64(389.3954513456468),
 np.float64(786.8713213271591),
 np.float64(289.22229416776),
 np.float64(2225.4153775915665),
 np.float64(nan),
 np.float64(622.061340539247),
 np.float64(5781.587094152921),
 np.float64(463.436629621918),
 np.float64(345.6862465136691),
 np.float64(1951.0769322706806),
 np.float64(356.9671143315726),
 np.float64(629.1944981629292),
 np.float64(265.33714917923504),
 np.float64(140.72053018211642),
 np.float64(701.8102397382395),
 np.float64(524.1137219369749),
 np.float64(2999.1280926410973),
 np.float64(296.9912671763162),
 np.float64(391.642008

In [98]:
summary_refstar_fluxes = { obs_file: {'mean': np.nanmean(fluxes), 'std': np.nanstd(fluxes)} for obs_file, fluxes in reference_star_fluxes_reduced.items()}
means_refstar_fluxes = [summary['mean'] for summary in summary_refstar_fluxes.values()]


In [97]:
summary_refstar_fluxes

{'2025-11-25-h02-m00-s14-ms936-tfn.fits': {'mean': np.float64(8574.068670257093),
  'std': np.float64(9738.304592996566)},
 '2025-11-25-h05-m00-s13-ms688-lsc.fits': {'mean': np.float64(8945.207228664522),
  'std': np.float64(10107.795797294079)},
 '2025-11-25-h05-m36-s42-ms454-lsc.fits': {'mean': np.float64(9245.973076514634),
  'std': np.float64(10440.638945741204)},
 '2025-11-25-h06-m06-s14-ms909-lsc.fits': {'mean': np.float64(9020.520877950834),
  'std': np.float64(10171.158663225173)},
 '2025-11-25-h07-m45-s12-ms840-elp.fits': {'mean': np.float64(7947.249511605864),
  'std': np.float64(9183.006412998122)},
 '2025-11-25-h08-m30-s09-ms718-elp.fits': {'mean': np.float64(8202.81822797935),
  'std': np.float64(9274.36219225685)},
 '2025-11-25-h09-m06-s12-ms841-elp.fits': {'mean': np.float64(8138.715508978118),
  'std': np.float64(9460.558404893707)},
 '2025-11-25-h10-m06-s14-ms367-elp.fits': {'mean': np.float64(7778.380876792169),
  'std': np.float64(9045.673614257841)},
 '2025-11-25-h1

In [104]:
#summary statistics of means_refstar_fluxes
import numpy as np
np.mean(means_refstar_fluxes), np.std(means_refstar_fluxes), np.min(means_refstar_fluxes), np.max(means_refstar_fluxes), np.median(means_refstar_fluxes)

(np.float64(8274.917734207396),
 np.float64(1683.6967296989392),
 np.float64(869.0441499013841),
 np.float64(11501.28109124161),
 np.float64(8463.226156961466))

In [113]:
means_refstar_fluxes

[np.float64(8574.068670257093),
 np.float64(8945.207228664522),
 np.float64(9245.973076514634),
 np.float64(9020.520877950834),
 np.float64(7947.249511605864),
 np.float64(8202.81822797935),
 np.float64(8138.715508978118),
 np.float64(7778.380876792169),
 np.float64(7580.723330709289),
 np.float64(9324.817005094754),
 np.float64(9280.928740976311),
 np.float64(9028.90664603481),
 np.float64(8115.442906960571),
 np.float64(7598.755751669499),
 np.float64(9324.29766126599),
 np.float64(9088.002778579887),
 np.float64(8205.112886458239),
 np.float64(4460.874268935698),
 np.float64(869.0441499013841),
 np.float64(9357.896514809467),
 np.float64(11501.28109124161),
 np.float64(7956.5380995165715),
 np.float64(8051.110904276024),
 np.float64(9614.73915983251),
 np.float64(9332.376890732192),
 np.float64(9098.697101410635),
 np.float64(9132.394470964546),
 np.float64(7039.242178050839),
 np.float64(6979.823745892228),
 np.float64(6908.586123355629),
 np.float64(8481.392004334988),
 np.float64

In [114]:
normalization_factor_to_themedian = [np.median(means_refstar_fluxes)/mean_flux for mean_flux in means_refstar_fluxes]
normalization_factor_to_themedian
#there is 1 datapoint with abnormally low mean flux, might need to remove it

[np.float64(0.987072355312463),
 np.float64(0.9461185124746391),
 np.float64(0.9153418560625711),
 np.float64(0.9382192305156587),
 np.float64(1.0649251850721546),
 np.float64(1.0317461537906423),
 np.float64(1.039872464841088),
 np.float64(1.088044708920416),
 np.float64(1.1164140660136195),
 np.float64(0.9076023853698636),
 np.float64(0.9118943150156299),
 np.float64(0.9373478416324338),
 np.float64(1.0428545002396115),
 np.float64(1.1137647311669463),
 np.float64(0.9076529369196893),
 np.float64(0.9312525934640999),
 np.float64(1.0314576135727782),
 np.float64(1.897212440148122),
 np.float64(9.738545685995172),
 np.float64(0.9043940744127563),
 np.float64(0.73585073608942),
 np.float64(1.06368197463614),
 np.float64(1.0511873774421072),
 np.float64(0.880234608164752),
 np.float64(0.9068671632160652),
 np.float64(0.9301580284114912),
 np.float64(0.9267258640512488),
 np.float64(1.2022922273296366),
 np.float64(1.2125272019859028),
 np.float64(1.2250301300218458),
 np.float64(0.997858